# Local Ollama Chatbot (Llama 3.1)

This notebook runs a minimal chatbot against a **local** Ollama model via its OpenAI-compatible API.

Prereq:
- Install Ollama and run it
- Ensure a model is pulled (example: `ollama pull llama3.1:latest`)

In [1]:
# Local backend setup (Ollama)
import os

# Ollama OpenAI-compatible base URL
os.environ.setdefault("LOCAL_OPENAI_BASE_URL", "http://localhost:11434/v1")

# Model to use (set LOCAL_MODEL_NAME to override)
MODEL_NAME = os.getenv("LOCAL_MODEL_NAME", "llama3.1_fine:latest")

# Optional: some gateways require a key; Ollama accepts any value.
# os.environ.setdefault("LOCAL_OPENAI_API_KEY", "ollama")

print("LOCAL_OPENAI_BASE_URL:", os.getenv("LOCAL_OPENAI_BASE_URL"))
print("MODEL_NAME:", MODEL_NAME)

LOCAL_OPENAI_BASE_URL: http://localhost:11434/v1
MODEL_NAME: llama3.1_fine:latest


In [2]:
# Connectivity check: local Ollama OpenAI-compatible API
import os, requests

base = os.getenv("LOCAL_OPENAI_BASE_URL", "http://localhost:11434/v1").rstrip("/")
url = f"{base}/models"
r = requests.get(url, timeout=10)
print("GET", url, "->", r.status_code)
print((r.text or "")[:300])

GET http://localhost:11434/v1/models -> 200
{"object":"list","data":[{"id":"llama3.1_fine:latest","object":"model","created":1768215487,"owned_by":"library"},{"id":"llama3.1:latest","object":"model","created":1767698163,"owned_by":"library"}]}



In [3]:
# Test local chat completions (optional)
import os, requests, json

base = os.getenv("LOCAL_OPENAI_BASE_URL", "http://localhost:11434/v1").rstrip("/")
url = f"{base}/chat/completions"
payload = {
    "model": MODEL_NAME,
    "messages": [{"role": "user", "content": "Say hello in one sentence."}],
    "max_tokens": 32,
    "temperature": 0.2,
}
headers = {"Content-Type": "application/json"}
resp = requests.post(url, headers=headers, json=payload, timeout=60)
print("POST", url, "->", resp.status_code)
print((resp.text or "")[:300])

POST http://localhost:11434/v1/chat/completions -> 200
{"id":"chatcmpl-361","object":"chat.completion","created":1768219200,"model":"llama3.1_fine:latest","system_fingerprint":"fp_ollama","choices":[{"index":0,"message":{"role":"assistant","content":"Hello! How can I assist you today?"},"finish_reason":"stop"}],"usage":{"prompt_tokens":16,"completion_to


In [4]:
import importlib, os

import deepseek_chat
importlib.reload(deepseek_chat)

from deepseek_chat import DeepSeekChatbot

bot = DeepSeekChatbot(model=MODEL_NAME)
print("Ready.")
print("LOCAL_OPENAI_BASE_URL:", os.getenv("LOCAL_OPENAI_BASE_URL"))
print("MODEL_NAME:", MODEL_NAME)
print("Try: bot.reply('Hello!')")

Ready.
LOCAL_OPENAI_BASE_URL: http://localhost:11434/v1
MODEL_NAME: llama3.1_fine:latest
Try: bot.reply('Hello!')


In [5]:
# Smoke test
print(bot.reply("Say hello in one sentence."))

Hello there! How may I assist you today?


# Finetuned model evaluation (same training prompts)

This section evaluates the fine-tuned Ollama model (`llama3.1_fine`) on the **same 3 tasks** used during training:

1. `quiz_generate` (schema check + exact-match vs dataset)
2. `grade_truth_answer` (score match vs dataset)
3. `rule_application_grade` (score match vs dataset)

Important: the prompts below copy the **exact** `SYSTEM_PROMPT_*` and `TASK:`-formatted user templates from the fine-tuning notebook.

In [5]:
# --- Prompt templates copied from fine-tuning notebook ---
from __future__ import annotations

import json
import random
from pathlib import Path
from typing import Any, Dict, List, Tuple


SYSTEM_PROMPT_QUIZ = (
    "You generate COLREGS quiz items. "
    "Use ONLY the provided quotes as evidence; do not invent rule text."
 )

SYSTEM_PROMPT_GRADE_TRUTH = (
    "You are a strict grader for COLREGS answers. "
    "Grade the STUDENT_ANSWER against the TRUTH_ANSWER using the RUBRIC. "
    "Do not add external facts."
 )

SYSTEM_PROMPT_GRADE_SCENARIO = (
    "You are a strict grader for COLREGS scenario rule-application. "
    "Grade the STUDENT_ANSWER against the REFERENCE_APPLICATION text. "
    "Do not add external facts."
 )


def _find_existing_path(*candidates: str) -> Path:
    for c in candidates:
        p = Path(c)
        if p.exists():
            return p.resolve()
    raise FileNotFoundError(f"None of these paths exist: {candidates}")


def _format_quotes(quotes: list[dict]) -> str:
    if not quotes:
        return "[No quotes provided]"
    lines = []
    for q in quotes:
        if not isinstance(q, dict):
            continue
        ref = (q.get("ref") or "").strip()
        qt = (q.get("quote") or "").strip()
        if qt:
            if ref:
                lines.append(f"- ({ref}) {qt}")
            else:
                lines.append(f"- {qt}")
    return "\n".join(lines) if lines else "[No quotes provided]"


def _normalize_text(s: str) -> str:
    return " ".join((s or "").split()).strip()


def _extract_first_json_object(text: str) -> dict[str, Any]:
    """Best-effort parse: accept raw JSON, or extract first {...} block."""
    s = (text or "").strip()
    if not s:
        raise ValueError("Empty model output")
    try:
        obj = json.loads(s)
        if isinstance(obj, dict):
            return obj
    except Exception:
        pass
    start = s.find("{")
    end = s.rfind("}")
    if start >= 0 and end > start:
        candidate = s[start : end + 1]
        obj = json.loads(candidate)
        if isinstance(obj, dict):
            return obj
    raise ValueError("Could not parse JSON object from model output")


def build_user_prompt_quiz(r: dict[str, Any]) -> str:
    quotes = r.get("supporting_quotes") or []
    if not isinstance(quotes, list):
        quotes = []
    difficulty = (r.get("difficulty") or "").strip()
    rule_refs = r.get("rule_refs") or []
    if not isinstance(rule_refs, list):
        rule_refs = []
    return (
        "TASK: quiz_generate\n"
        + (f"DIFFICULTY: {difficulty}\n" if difficulty else "")
        + ("RULE_REFS: " + ", ".join([str(x) for x in rule_refs if str(x).strip()]) + "\n" if rule_refs else "")
        + "QUOTES:\n" + _format_quotes(quotes) + "\n\n"
        + "Generate ONE quiz item as JSON with keys: question, truth_answer."
    )


def build_user_prompt_grade_truth(r: dict[str, Any], ua: dict[str, Any]) -> str:
    q = (r.get("question") or "").strip()
    truth = (r.get("truth_answer") or "").strip()
    rubric = r.get("rubric") or []
    support = r.get("supporting_quotes") or []
    if not isinstance(rubric, list):
        rubric = []
    if not isinstance(support, list):
        support = []
    student = (ua.get("answer") or "").strip()

    rubric_lines: list[str] = []
    for it in rubric:
        if not isinstance(it, dict):
            continue
        rid = (it.get("id") or "").strip()
        mx = it.get("max")
        crit = (it.get("criterion") or "").strip()
        if rid and crit and isinstance(mx, (int, float)):
            rubric_lines.append(f"- {rid} (max {int(mx)}): {crit}")

    return (
        "TASK: grade_truth_answer\n"
        + "QUESTION:\n" + q + "\n\n"
        + "TRUTH_ANSWER:\n" + truth + "\n\n"
        + "RUBRIC (sum to 10):\n" + "\n".join(rubric_lines) + "\n\n"
        + "SUPPORTING_QUOTES:\n" + _format_quotes(support) + "\n\n"
        + "STUDENT_ANSWER:\n" + student + "\n\n"
        + "Return ONLY JSON in the schema: {\"score_total\": int, \"breakdown\": [...], \"missing_points\": [...], \"incorrect_claims\": [...], \"feedback\": str}."
    )


def build_user_prompt_grade_scenario(r: dict[str, Any]) -> str:
    # scenario_dataset.json includes the reference text already
    scenario = (r.get("scenario") or "").strip()
    source_id = str(r.get("source_id") or "").strip()
    rule_refs = r.get("rule_refs") or []
    if not isinstance(rule_refs, list):
        rule_refs = []
    reference_application = (r.get("reference_application") or "").strip()
    student = (r.get("student_answer") or "").strip()
    return (
        "TASK: rule_application_grade\n"
        + f"SOURCE_ID: {source_id}\n"
        + ("RULE_REFS: " + ", ".join([str(x) for x in rule_refs if str(x).strip()]) + "\n" if rule_refs else "")
        + "SCENARIO:\n" + scenario + "\n\n"
        + "REFERENCE_APPLICATION:\n" + reference_application + "\n\n"
        + "STUDENT_ANSWER:\n" + student + "\n\n"
        + "Return ONLY JSON: {\"score\": int} (0..10)."
    )


def _load_json(path: Path) -> Any:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


# Locate datasets (this notebook lives in Finetune/, but CWD can vary)
QUIZ_JSON = _find_existing_path(
    "quizz_gen.json",
    str(Path("Finetune") / "quizz_gen.json"),
    str(Path("..") / "Finetune" / "quizz_gen.json"),
    r"C:\Users\celli\Documents\.PIP2026\PIP-2026-LOTUSim_G5_Fatigue\Finetune\quizz_gen.json",
)
GRADE_JSON = _find_existing_path(
    "grade_dataset.json",
    str(Path("Finetune") / "grade_dataset.json"),
    str(Path("..") / "Finetune" / "grade_dataset.json"),
    r"C:\Users\celli\Documents\.PIP2026\PIP-2026-LOTUSim_G5_Fatigue\Finetune\grade_dataset.json",
)
SCENARIO_JSON = _find_existing_path(
    "scenario_dataset.json",
    str(Path("Finetune") / "scenario_dataset.json"),
    str(Path("..") / "Finetune" / "scenario_dataset.json"),
    r"C:\Users\celli\Documents\.PIP2026\PIP-2026-LOTUSim_G5_Fatigue\Finetune\scenario_dataset.json",
)

print("Quiz:", QUIZ_JSON)
print("Grade:", GRADE_JSON)
print("Scenario:", SCENARIO_JSON)

quiz_rows = _load_json(Path(QUIZ_JSON))
grade_rows = _load_json(Path(GRADE_JSON))
scenario_rows = _load_json(Path(SCENARIO_JSON))

assert isinstance(quiz_rows, list)
assert isinstance(grade_rows, list)
assert isinstance(scenario_rows, list)
print("Loaded:", len(quiz_rows), "quiz;", len(grade_rows), "grade;", len(scenario_rows), "scenario")

Quiz: C:\Users\celli\Documents\.PIP2026\PIP-2026-LOTUSim_G5_Fatigue\Finetune\quizz_gen.json
Grade: C:\Users\celli\Documents\.PIP2026\PIP-2026-LOTUSim_G5_Fatigue\Finetune\grade_dataset.json
Scenario: C:\Users\celli\Documents\.PIP2026\PIP-2026-LOTUSim_G5_Fatigue\Finetune\scenario_dataset.json
Loaded: 399 quiz; 192 grade; 291 scenario


In [6]:
# Load scenario reference text (ID -> Application_Regle)
SCENARIO_SOURCE_JSON = _find_existing_path(
    "colreg_v9_split.json",
    str(Path("Finetune") / "colreg_v9_split.json"),
    str(Path("..") / "Finetune" / "colreg_v9_split.json"),
    r"C:\Users\celli\Documents\.PIP2026\PIP-2026-LOTUSim_G5_Fatigue\Finetune\colreg_v9_split.json",
)
scenario_source_rows = _load_json(Path(SCENARIO_SOURCE_JSON))
assert isinstance(scenario_source_rows, list)

application_by_id: dict[str, str] = {}
for row in scenario_source_rows:
    if not isinstance(row, dict):
        continue
    sid = str(row.get("ID") or "").strip()
    app = (row.get("Application_Regle") or "").strip()
    if sid and app:
        application_by_id[sid] = app

print("Scenario sources:", len(application_by_id))

Scenario sources: 209


In [7]:
# --- Batch evaluator (calls Ollama via DeepSeekChatbot) ---
import math


def _call_model_json(*, system_prompt: str, user_prompt: str, max_tokens: int = 512) -> tuple[str, dict[str, Any] | None, str | None]:
    """Returns (raw_text, parsed_json_or_none, error_or_none)."""
    bot.reset()
    raw = bot.reply(
        user_prompt,
        system_prompt=system_prompt,
        max_tokens=max_tokens,
        temperature=0.0,
        top_p=1.0,
    )
    try:
        obj = _extract_first_json_object(raw)
        return raw, obj, None
    except Exception as e:
        return raw, None, str(e)


def evaluate_quiz_generate(*, n: int = 30, seed: int = 0) -> dict[str, Any]:
    rows = [r for r in quiz_rows if isinstance(r, dict)]
    random.Random(seed).shuffle(rows)
    rows = rows[: max(0, int(n))]

    ok = 0
    parse_fail = 0
    exact_match = 0
    samples: list[dict[str, Any]] = []

    for r in rows:
        user = build_user_prompt_quiz(r)
        raw, obj, err = _call_model_json(system_prompt=SYSTEM_PROMPT_QUIZ, user_prompt=user, max_tokens=256)
        if obj is None:
            parse_fail += 1
            samples.append({"task": "quiz_generate", "error": err, "raw": raw[:800]})
            continue
        ok += 1
        q_pred = _normalize_text(str(obj.get("question", "")))
        t_pred = _normalize_text(str(obj.get("truth_answer", "")))
        q_gt = _normalize_text(str(r.get("question", "")))
        t_gt = _normalize_text(str(r.get("truth_answer", "")))
        if q_pred == q_gt and t_pred == t_gt:
            exact_match += 1
        if len(samples) < 3:
            samples.append({
                "task": "quiz_generate",
                "gt": {"question": q_gt, "truth_answer": t_gt},
                "pred": {"question": q_pred, "truth_answer": t_pred},
            })

    total = len(rows)
    return {
        "total": total,
        "parsed_ok": ok,
        "parse_fail": parse_fail,
        "exact_match": exact_match,
        "exact_match_rate": (exact_match / total) if total else None,
        "samples": samples,
    }


def evaluate_grade_truth_answer(*, n_questions: int = 50, seed: int = 0) -> dict[str, Any]:
    rows = [r for r in grade_rows if isinstance(r, dict)]
    random.Random(seed).shuffle(rows)
    rows = rows[: max(0, int(n_questions))]

    total = 0
    parsed_ok = 0
    parse_fail = 0
    score_match = 0
    abs_errs: list[float] = []
    samples: list[dict[str, Any]] = []

    for r in rows:
        uas = r.get("user_answers") or []
        if not isinstance(uas, list) or not uas:
            continue
        # pick one answer per question to keep eval cost reasonable
        ua = random.Random(seed + total).choice([x for x in uas if isinstance(x, dict)] or [{}])
        grade = ua.get("grade") or {}
        if not isinstance(grade, dict):
            continue
        gt_score = grade.get("score_total")
        if not isinstance(gt_score, int):
            continue

        user = build_user_prompt_grade_truth(r, ua)
        raw, obj, err = _call_model_json(system_prompt=SYSTEM_PROMPT_GRADE_TRUTH, user_prompt=user, max_tokens=512)
        total += 1
        if obj is None:
            parse_fail += 1
            if len(samples) < 3:
                samples.append({"task": "grade_truth_answer", "gt_score_total": gt_score, "error": err, "raw": raw[:800]})
            continue
        parsed_ok += 1
        pred_score = obj.get("score_total")
        if isinstance(pred_score, bool):
            pred_score = int(pred_score)
        if isinstance(pred_score, (int, float)) and float(pred_score).is_integer():
            pred_score = int(pred_score)
        else:
            pred_score = None
        if pred_score is not None:
            if pred_score == gt_score:
                score_match += 1
            abs_errs.append(abs(float(pred_score) - float(gt_score)))
        if len(samples) < 3:
            samples.append({"task": "grade_truth_answer", "gt_score_total": gt_score, "pred_score_total": pred_score})

    mae = (sum(abs_errs) / len(abs_errs)) if abs_errs else None
    return {
        "total": total,
        "parsed_ok": parsed_ok,
        "parse_fail": parse_fail,
        "score_match": score_match,
        "score_match_rate": (score_match / total) if total else None,
        "mae_score_total": mae,
        "samples": samples,
    }


def _flatten_scenario_rows() -> list[dict[str, Any]]:
    flat: list[dict[str, Any]] = []
    for r in scenario_rows:
        if not isinstance(r, dict):
            continue
        scenario = (r.get("scenario") or "").strip()
        source_id = str(r.get("source_id") or "").strip()
        rule_refs = r.get("rule_refs") or []
        if not isinstance(rule_refs, list):
            rule_refs = []
        ref_app = (r.get("reference_application") or "").strip()
        answers = r.get("answers") or []
        if not scenario or not source_id or not ref_app or not isinstance(answers, list):
            continue
        for a in answers:
            if not isinstance(a, dict):
                continue
            student = (a.get("answer") or "").strip()
            score = a.get("score")
            if not student or not isinstance(score, int):
                continue
            flat.append({
                "scenario": scenario,
                "source_id": source_id,
                "rule_refs": rule_refs,
                "reference_application": ref_app,
                "student_answer": student,
                "score": score,
            })
    return flat


def evaluate_rule_application_grade(*, n: int = 120, seed: int = 0) -> dict[str, Any]:
    rows = _flatten_scenario_rows()
    random.Random(seed).shuffle(rows)
    rows = rows[: max(0, int(n))]

    total = 0
    parsed_ok = 0
    parse_fail = 0
    score_match = 0
    abs_errs: list[float] = []
    samples: list[dict[str, Any]] = []

    for r in rows:
        user = build_user_prompt_grade_scenario(r)
        raw, obj, err = _call_model_json(system_prompt=SYSTEM_PROMPT_GRADE_SCENARIO, user_prompt=user, max_tokens=128)
        total += 1
        if obj is None:
            parse_fail += 1
            if len(samples) < 3:
                samples.append({"task": "rule_application_grade", "gt_score": r["score"], "error": err, "raw": raw[:800]})
            continue
        parsed_ok += 1
        pred_score = obj.get("score")
        if isinstance(pred_score, bool):
            pred_score = int(pred_score)
        if isinstance(pred_score, (int, float)) and float(pred_score).is_integer():
            pred_score = int(pred_score)
        else:
            pred_score = None
        if pred_score is not None:
            if pred_score == r["score"]:
                score_match += 1
            abs_errs.append(abs(float(pred_score) - float(r["score"])))
        if len(samples) < 3:
            samples.append({"task": "rule_application_grade", "gt_score": r["score"], "pred_score": pred_score})

    mae = (sum(abs_errs) / len(abs_errs)) if abs_errs else None
    return {
        "total": total,
        "parsed_ok": parsed_ok,
        "parse_fail": parse_fail,
        "score_match": score_match,
        "score_match_rate": (score_match / total) if total else None,
        "mae_score": mae,
        "samples": samples,
    }

In [8]:
# Override scenario eval to match training prompt (inject REFERENCE_APPLICATION from colreg_v9_split.json)
def _flatten_scenario_rows() -> list[dict[str, Any]]:
    flat: list[dict[str, Any]] = []
    for r in scenario_rows:
        if not isinstance(r, dict):
            continue
        scenario = (r.get("scenario") or "").strip()
        source_id_raw = r.get("source_id")
        source_id = str(source_id_raw).strip() if source_id_raw is not None else ""
        rule_refs = r.get("rule_refs") or []
        if not isinstance(rule_refs, list):
            rule_refs = []
        answers = r.get("answers") or []
        if not scenario or not source_id or not isinstance(answers, list):
            continue
        ref_app = (application_by_id.get(source_id) or "").strip()
        if not ref_app:
            continue
        for a in answers:
            if not isinstance(a, dict):
                continue
            student = (a.get("answer") or "").strip()
            score = a.get("score")
            if not student or not isinstance(score, int):
                continue
            flat.append({
                "scenario": scenario,
                "source_id": source_id,
                "rule_refs": rule_refs,
                "reference_application": ref_app,
                "student_answer": student,
                "score": score,
            })
    return flat

In [9]:
# Run evaluation (adjust sample sizes if needed)
quiz_report = evaluate_quiz_generate(n=30, seed=0)
grade_report = evaluate_grade_truth_answer(n_questions=40, seed=0)
scenario_report = evaluate_rule_application_grade(n=120, seed=0)

print("=== quiz_generate ===")
print({k: v for k, v in quiz_report.items() if k != "samples"})
print("samples:")
for s in quiz_report["samples"]:
    print(s)

print("\n=== grade_truth_answer ===")
print({k: v for k, v in grade_report.items() if k != "samples"})
print("samples:")
for s in grade_report["samples"]:
    print(s)

print("\n=== rule_application_grade ===")
print({k: v for k, v in scenario_report.items() if k != "samples"})
print("samples:")
for s in scenario_report["samples"]:
    print(s)

=== quiz_generate ===
{'total': 30, 'parsed_ok': 30, 'parse_fail': 0, 'exact_match': 0, 'exact_match_rate': 0.0}
samples:
{'task': 'quiz_generate', 'gt': {'question': 'What whistle signal indicates intent to overtake on the port side in a narrow channel?', 'truth_answer': 'Two prolonged blasts followed by two short blasts.'}, 'pred': {'question': 'What signal indicates a vessel intends to overtake another on the port side?', 'truth_answer': 'Two prolonged blasts followed by two short blasts.'}}
{'task': 'quiz_generate', 'gt': {'question': 'How are three vertical lights spaced?', 'truth_answer': 'Equally spaced.'}, 'pred': {'question': 'How far apart should three masthead lights be?', 'truth_answer': 'Equally spaced.'}}
{'task': 'quiz_generate', 'gt': {'question': 'What defines a vessel constrained by her draught?', 'truth_answer': 'A power-driven vessel which because of her draught in relation to the available depth and width of navigable water is severely restricted in her ability to 

In [15]:
# --- Interactive quiz (4 questions) + grading ---
# Uses the same TASK: prompts and SYSTEM_PROMPT_* as training/evaluation above.

from typing import Optional
import sys


QUIZ4_TOPICS = [
    "Rule 5 (Look-out)",
    "Rule 6 (Safe speed)",
    "Rule 7 (Risk of collision)",
    "Rule 8 (Action to avoid collision)",
    "Rule 13 (Overtaking)",
    "Rule 14 (Head-on situation)",
    "Rule 15 (Crossing situation)",
    "Rule 16 (Action by give-way vessel)",
    "Rule 17 (Action by stand-on vessel)",
    "Rule 19 (Conduct of vessels in restricted visibility)",
 ]


QUIZ_RUBRIC = [
    {
        "id": "R1",
        "max": 5,
        "criterion": "Correctly identifies the relevant rule(s) and who must keep out of the way / stand on (if applicable).",
    },
    {
        "id": "R2",
        "max": 5,
        "criterion": "States the correct required actions/precautions (early & substantial action, avoid crossing ahead, safe speed, look-out, etc.) without adding incorrect claims.",
    },
 ]


def _maybe_supporting_quotes(query: str, *, k: int = 6, max_chars: int = 420) -> list[dict]:
    """Return [{'ref','quote'}] from attached PDF if available; else []."""
    try:
        if "ATTACHED_PDF" not in globals():
            return []
        pdf = globals().get("ATTACHED_PDF")
        if pdf is None:
            return []
        if "retrieve_context_multiquery" not in globals():
            return []
        retr = globals()["retrieve_context_multiquery"]
        indices = retr(query, k=min(int(k), 10))
        chunks = getattr(pdf, "chunks", None)
        if not isinstance(indices, list) or chunks is None:
            return []
        out: list[dict] = []
        for idx in indices[:k]:
            try:
                ch = chunks[int(idx)]
                txt = str(getattr(ch, "text", "") or "").strip()
                if not txt:
                    continue
                out.append({"ref": f"Chunk {int(idx)+1}", "quote": txt[:max_chars]})
            except Exception:
                continue
        return out
    except Exception:
        return []


def _generate_one_quiz_item(*, topic: str, difficulty: str = "") -> dict[str, str]:
    quotes = _maybe_supporting_quotes(topic)
    user = (
        "TASK: quiz_generate\n"
        + (f"DIFFICULTY: {difficulty}\n" if difficulty else "")
        + f"RULE_REFS: {topic}\n"
        + "QUOTES:\n" + _format_quotes(quotes) + "\n\n"
        + "Generate ONE quiz item as JSON with keys: question, truth_answer."
    )
    raw, obj, err = _call_model_json(system_prompt=SYSTEM_PROMPT_QUIZ, user_prompt=user, max_tokens=256)
    if obj is None:
        raise RuntimeError(f"Model did not return JSON: {err}. Raw: {raw[:200]}")
    q = str(obj.get("question", "")).strip()
    t = str(obj.get("truth_answer", "")).strip()
    if not q or not t:
        raise RuntimeError(f"Missing keys in model JSON. Got: {obj}")
    return {"topic": topic, "question": q, "truth_answer": t}


def generate_quiz4(*, seed: int = 0) -> list[dict[str, str]]:
    rng = random.Random(seed)
    topics = QUIZ4_TOPICS[:]
    rng.shuffle(topics)
    picked = topics[:4]
    difficulties = ["easy", "medium", "medium", "hard"]
    quiz: list[dict[str, str]] = []
    for i, topic in enumerate(picked):
        diff = difficulties[i] if i < len(difficulties) else ""
        # Retry once if the model fails to emit valid JSON
        try:
            quiz.append(_generate_one_quiz_item(topic=topic, difficulty=diff))
        except Exception:
            quiz.append(_generate_one_quiz_item(topic=topic, difficulty=diff))
    return quiz


def grade_user_answer(*, question: str, truth_answer: str, user_answer: str) -> dict[str, Any]:
    support = _maybe_supporting_quotes(question + "\n" + truth_answer, k=6)
    r = {
        "question": question,
        "truth_answer": truth_answer,
        "rubric": QUIZ_RUBRIC,
        "supporting_quotes": support,
    }
    ua = {"answer": user_answer}
    prompt = build_user_prompt_grade_truth(r, ua)
    raw, obj, err = _call_model_json(system_prompt=SYSTEM_PROMPT_GRADE_TRUTH, user_prompt=prompt, max_tokens=512)
    if obj is None:
        raise RuntimeError(f"Model did not return JSON: {err}. Raw: {raw[:300]}")
    # Normalize score_total
    score_total = obj.get("score_total")
    if isinstance(score_total, bool):
        score_total = int(score_total)
    if isinstance(score_total, (int, float)) and float(score_total).is_integer():
        score_total = int(score_total)
    else:
        score_total = None
    if score_total is None:
        raise RuntimeError(f"Missing/invalid score_total in: {obj}")
    score_total = max(0, min(10, int(score_total)))
    feedback = str(obj.get("feedback", "")).strip()
    return {"score_total": score_total, "feedback": feedback, "raw": obj}


def run_quiz4(*, seed: int = 0) -> None:
    if not globals().get("MODEL_NAME"):
        raise RuntimeError("MODEL_NAME not set")
    if not globals().get("bot"):
        raise RuntimeError("bot not initialized")
    if not _maybe_supporting_quotes("Rule 5", k=1):
        print("Note: No PDF context attached; QUOTES/SUPPORTING_QUOTES will be empty.", flush=True)
        print("For best grounding, run the PDF attachment section and attach the COLREG PDF first.", flush=True)
        print("", flush=True)
    quiz = generate_quiz4(seed=seed)
    total = 0
    print("--- COLREGS Quiz (4 questions) ---", flush=True)
    print("Type /exit to stop.\n", flush=True)
    for i, item in enumerate(quiz, start=1):
        q = item["question"]
        truth = item["truth_answer"]
        topic = item.get("topic", "")
        print(f"Q{i} ({topic}): {q}", flush=True)
        try:
            # Jupyter/VS Code can buffer prints; flush before waiting for input.
            sys.stdout.flush()
            ans = input("Your answer> ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\n(quiz aborted)")
            return
        if ans.lower().strip() in {"/exit", "/quit"}:
            print("(quiz aborted)")
            return
        try:
            g = grade_user_answer(question=q, truth_answer=truth, user_answer=ans)
            total += int(g.get("score_total", 0) or 0)
            print(f"Score: {g['score_total']}/10", flush=True)
            if g.get("feedback"):
                print("Feedback:", g["feedback"], flush=True)
        except Exception as e:
            print("Grading failed:", e, flush=True)
        print("", flush=True)
    print(f"Total: {total}/40", flush=True)

In [16]:
# Start the interactive quiz
run_quiz4(seed=0)

--- COLREGS Quiz (4 questions) ---
Type /exit to stop.

Q1 (Rule 16 (Action by give-way vessel)): What must a give-way vessel do?
Score: 5/10
Feedback: Basic requirement correct but misses the 'early and substantial' aspect.

Q2 (Rule 17 (Action by stand-on vessel)): What must a stand-on vessel do when the give-way vessel fails to act?
Score: 0/10
Feedback: Incorrect; Rule 17(a)(ii) allows stand-on vessel to take action alone if give-way vessel does not comply.

Q3 (Rule 6 (Safe speed)): What must a vessel using a Traffic Separation Scheme do?
Score: 10/10
Feedback: Complete answer covering all requirements of Rule 10.

Q4 (Rule 14 (Head-on situation)): What must each vessel do in a head-on situation?
Score: 6/10
Feedback: Basic concept is correct but lacks specificity about which vessel gives way and what course alteration is required.

Total: 21/40


# Old

In [12]:
# --- PDF attachment (Hybrid RAG: BM25 + Embeddings + Multi-Query + CoVe) ---
from __future__ import annotations

import io
import re
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

# Install / import PDF extraction
try:
    from pypdf import PdfReader  # type: ignore
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "pypdf"])
    from pypdf import PdfReader  # type: ignore

# Widgets (optional UI)
try:
    import ipywidgets as widgets  # type: ignore
    from IPython.display import display  # type: ignore
except Exception:
    widgets = None
    display = None

# Hybrid retrieval deps
import numpy as np
try:
    from rank_bm25 import BM25Okapi  # type: ignore
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "rank-bm25"])
    from rank_bm25 import BM25Okapi  # type: ignore

try:
    from sentence_transformers import SentenceTransformer  # type: ignore
except Exception:
    SentenceTransformer = None  # type: ignore


_WORD_RE = re.compile(r"\b\w+\b", re.UNICODE)
_STOPWORDS = {
    # EN
    "the","a","an","and","or","to","of","in","on","for","with","is","are","was","were","be","as","at","by","it","this","that","from","into","over","under","not",
    # FR
    "le","la","les","un","une","des","et","ou","de","du","dans","sur","pour","avec","est","sont","été","être","ce","cet","cette","ces","pas","plus",
}


def _tokenize(text: str) -> List[str]:
    words = [w.lower() for w in _WORD_RE.findall(text)]
    return [w for w in words if len(w) >= 2 and w not in _STOPWORDS]


def _pdf_bytes_to_text(data: bytes) -> str:
    reader = PdfReader(io.BytesIO(data))
    parts: List[str] = []
    for page in reader.pages:
        try:
            parts.append(page.extract_text() or "")
        except Exception:
            parts.append("")
    return "\n".join(parts).strip()


def load_pdf_from_path(path: str) -> str:
    with open(path, "rb") as f:
        return _pdf_bytes_to_text(f.read())


_COLREG_HEAD_RE = re.compile(
    r"(?im)^(rule\s+\d+\b.*|annex\s+[ivx]+\b.*|appendix\b.*)$"
 )


def _normalize_ws(text: str) -> str:
    return re.sub(r"\s+", " ", text).strip()


def _slice_tokens(text: str, *, max_tokens: int, overlap_tokens: int) -> List[str]:
    """Approx token-based slicing (whitespace tokens)."""
    text = _normalize_ws(text)
    if not text:
        return []
    toks = text.split()
    if not toks:
        return []
    out: List[str] = []
    start = 0
    n = len(toks)
    max_tokens = max(1, int(max_tokens))
    overlap_tokens = max(0, int(overlap_tokens))
    while start < n:
        end = min(n, start + max_tokens)
        out.append(" ".join(toks[start:end]))
        if end >= n:
            break
        start = max(0, end - overlap_tokens)
    return out


def chunk_colreg(
    text: str,
    *,
    chunk_tokens: int = 500,
    overlap_tokens: int = 150,
) -> List[Tuple[str, str]]:
    """Chunk COLREG-like docs by headings (Rule/Annex), then slice each section.

    Requirement mapping:
    - chunk size ~500 tokens
    - overlap ~150 tokens
    """
    raw = text.replace("\r\n", "\n")
    lines = [ln.strip() for ln in raw.split("\n")]
    headings: List[Tuple[int, str]] = []
    for i, ln in enumerate(lines):
        if not ln:
            continue
        m = _COLREG_HEAD_RE.match(ln)
        if m:
            headings.append((i, m.group(1).strip()))
    if not headings:
        joined = _normalize_ws(raw)
        return [("Document", ch) for ch in _slice_tokens(joined, max_tokens=chunk_tokens, overlap_tokens=overlap_tokens)]

    chunks: List[Tuple[str, str]] = []
    for idx, (start_i, title) in enumerate(headings):
        end_i = headings[idx + 1][0] if idx + 1 < len(headings) else len(lines)
        section_text = _normalize_ws(" ".join([ln for ln in lines[start_i:end_i] if ln]))
        if not section_text:
            continue
        for ch in _slice_tokens(section_text, max_tokens=chunk_tokens, overlap_tokens=overlap_tokens):
            chunks.append((title, ch))
    return chunks


def _minmax(scores: np.ndarray) -> np.ndarray:
    if scores.size == 0:
        return scores
    mn = float(scores.min())
    mx = float(scores.max())
    if mx - mn < 1e-9:
        return np.zeros_like(scores, dtype=float)
    return (scores - mn) / (mx - mn)


@dataclass
class Chunk:
    idx: int
    section: str
    text: str


@dataclass
class AttachedPDF:
    name: str
    text: str
    chunks: List[Chunk]
    bm25: BM25Okapi
    bm25_tokens: List[List[str]]
    embedder_name: str
    embeddings: Optional[np.ndarray]  # shape: (n, d) or None


ATTACHED_PDF: AttachedPDF | None = None
RETRIEVE_K = 10
BM25_WEIGHT = 0.55
SEM_WEIGHT = 0.45
EMBEDDER_NAME = "sentence-transformers/all-MiniLM-L6-v2"
CHUNK_TOKENS = 1024
CHUNK_OVERLAP_TOKENS = 256
COVE_ENABLED = True
_EMBEDDER: Optional[SentenceTransformer] = None  # type: ignore


def _get_embedder() -> Optional["SentenceTransformer"]:
    global _EMBEDDER
    if SentenceTransformer is None:
        return None
    if _EMBEDDER is None:
        _EMBEDDER = SentenceTransformer(EMBEDDER_NAME)
    return _EMBEDDER


def attach_pdf_text(name: str, text: str) -> None:
    """Attach PDF text and build hybrid indexes (BM25 + embeddings)."""
    global ATTACHED_PDF
    pairs = chunk_colreg(text, chunk_tokens=CHUNK_TOKENS, overlap_tokens=CHUNK_OVERLAP_TOKENS)
    chunks = [Chunk(idx=i, section=sec, text=ch) for i, (sec, ch) in enumerate(pairs)]
    bm25_tokens = [_tokenize(c.text) for c in chunks]
    bm25 = BM25Okapi(bm25_tokens)

    embedder = _get_embedder()
    embeddings: Optional[np.ndarray] = None
    if embedder is not None:
        docs = [f"{c.section}. {c.text}" for c in chunks]
        emb = embedder.encode(docs, normalize_embeddings=True, show_progress_bar=False)
        embeddings = np.asarray(emb, dtype=np.float32)

    ATTACHED_PDF = AttachedPDF(
        name=name,
        text=text,
        chunks=chunks,
        bm25=bm25,
        bm25_tokens=bm25_tokens,
        embedder_name=EMBEDDER_NAME,
        embeddings=embeddings,
    )
    print(f"Attached PDF: {name} ({len(text):,} chars, {len(chunks)} chunks)")
    print(f"Chunking: ~{CHUNK_TOKENS} tokens, overlap ~{CHUNK_OVERLAP_TOKENS} tokens")
    if embeddings is None:
        print("(Embeddings disabled: sentence-transformers not available)")


def clear_pdf() -> None:
    global ATTACHED_PDF
    ATTACHED_PDF = None
    print("PDF cleared")


def _hybrid_scores(question: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    assert ATTACHED_PDF is not None
    q_tokens = _tokenize(question)
    bm25_scores = np.asarray(ATTACHED_PDF.bm25.get_scores(q_tokens), dtype=float)
    bm25_norm = _minmax(bm25_scores)

    sem_norm = np.zeros_like(bm25_norm, dtype=float)
    if ATTACHED_PDF.embeddings is not None:
        embedder = _get_embedder()
        if embedder is not None:
            q_emb = embedder.encode([question], normalize_embeddings=True, show_progress_bar=False)
            q_emb = np.asarray(q_emb[0], dtype=np.float32)
            sem = np.dot(ATTACHED_PDF.embeddings, q_emb).astype(float)
            sem_norm = _minmax(sem)

    hybrid = BM25_WEIGHT * bm25_norm + SEM_WEIGHT * sem_norm
    return hybrid, bm25_norm, sem_norm


def _diverse_topk(indices: List[int], *, k: int) -> List[int]:
    """Keep a diverse set of chunks by section to avoid near-duplicates."""
    assert ATTACHED_PDF is not None
    picked: List[int] = []
    seen_sections: Dict[str, int] = {}
    for idx in indices:
        sec = ATTACHED_PDF.chunks[idx].section
        if seen_sections.get(sec, 0) >= 2:
            continue
        picked.append(idx)
        seen_sections[sec] = seen_sections.get(sec, 0) + 1
        if len(picked) >= k:
            break
    return picked


def retrieve_context(question: str, *, k: int = 6, pool: int = 20) -> List[int]:
    """Hybrid retrieval with diversity: returns chunk indices."""
    if not ATTACHED_PDF:
        return []
    hybrid, _, _ = _hybrid_scores(question)
    ranked = np.argsort(-hybrid)[: max(pool, k)].tolist()
    ranked = [i for i in ranked if hybrid[i] > 0]
    return _diverse_topk(ranked, k=k)


def _build_context(indices: List[int]) -> str:
    """Hierarchy-aware context formatting (grouped by section)."""
    assert ATTACHED_PDF is not None
    if not indices:
        return ""
    # Preserve retrieval order but group by section for readability/hierarchy clarity.
    by_section: Dict[str, List[int]] = {}
    section_order: List[str] = []
    for idx in indices:
        sec = ATTACHED_PDF.chunks[idx].section
        if sec not in by_section:
            by_section[sec] = []
            section_order.append(sec)
        by_section[sec].append(idx)

    blocks: List[str] = []
    for sec in section_order:
        blocks.append(f"=== {sec} ===")
        for idx in by_section[sec]:
            c = ATTACHED_PDF.chunks[idx]
            blocks.append(f"(Chunk {idx+1}) {c.text}")
        blocks.append("")
    return "\n".join(blocks).strip()


def _generate_subqueries(question: str) -> List[str]:
    """Multi-Query Retrieval: have the LLM generate 3 diverse sub-queries."""
    prompt = (
        "Generate exactly 3 short, diverse search sub-queries to retrieve passages from the COLREG document. "
        "Each sub-query should be a keyword-style query and may include Rule/Annex numbers if relevant.\n"
        "Rules:\n"
        "- Output ONLY the 3 queries, one per line\n"
        "- No explanations\n"
        "- Keep each query <= 12 words\n\n"
        f"User question: {question}"
    )
    raw = bot.reply(prompt, max_tokens=96, temperature=0.2)
    lines = [ln.strip(" -\t") for ln in raw.splitlines() if ln.strip()]
    # Deduplicate while preserving order
    seen = set()
    out: List[str] = []
    for ln in lines:
        key = ln.lower()
        if key in seen:
            continue
        seen.add(key)
        out.append(ln)
        if len(out) >= 3:
            break
    return out if out else [question]


def retrieve_context_multiquery(question: str, *, k: int = 6) -> List[int]:
    """Two-pass retrieval: initial retrieval + 3 LLM sub-queries -> merged scoring."""
    if not ATTACHED_PDF:
        return []
    # Pass 1 (seed)
    init = retrieve_context(question, k=max(4, k), pool=30)
    # Pass 2 (multi-query)
    queries = _generate_subqueries(question)
    merged_scores = np.zeros(len(ATTACHED_PDF.chunks), dtype=float)
    for q in [question] + queries:
        hybrid, _, _ = _hybrid_scores(q)
        merged_scores = np.maximum(merged_scores, hybrid)
    ranked = np.argsort(-merged_scores)[: 40].tolist()
    ranked = [i for i in ranked if merged_scores[i] > 0]
    # Ensure we keep some of the seed results
    for idx in init:
        if idx in ranked:
            ranked.remove(idx)
        ranked.insert(0, idx)
    return _diverse_topk(ranked, k=k)


def _cove_answer(question: str, *, context: str) -> str:
    """Chain-of-Verification (CoVe) without revealing internal reasoning: returns final answer only."""
    # 1) Draft answer
    draft_prompt = (
        "Answer the QUESTION using ONLY the EXCERPTS. "
        "Cite evidence like (Chunk 12). If the excerpts do not contain the answer, say you don't know.\n\n"
        f"EXCERPTS:\n{context if context else '[No relevant excerpts found]'}\n\n"
        f"QUESTION:\n{question}\n\n"
        "Output ONLY the draft answer."
    )
    draft = bot.reply(draft_prompt, max_tokens=512, temperature=0.2).strip()

    # 2) Verification questions
    ver_prompt = (
        "Create exactly 3 verification questions that would check whether the DRAFT is fully supported by the EXCERPTS. "
        "Focus on potential missing conditions, exceptions, or definitions.\n\n"
        f"QUESTION:\n{question}\n\n"
        f"DRAFT:\n{draft}\n\n"
        "Output ONLY the 3 verification questions, one per line."
    )
    ver_raw = bot.reply(ver_prompt, max_tokens=128, temperature=0.2)
    ver_qs = [ln.strip(" -\t") for ln in ver_raw.splitlines() if ln.strip()]
    ver_qs = ver_qs[:3] if ver_qs else []
    ver_q_block = "\n".join(f"- {q}" for q in ver_qs) if ver_qs else "- (none)"

    # 3) Answer verification questions strictly from excerpts
    ver_ans_prompt = (
        "Using ONLY the EXCERPTS, answer each verification question briefly. "
        "If an answer is not supported by the EXCERPTS, write 'Not supported'.\n\n"
        f"EXCERPTS:\n{context if context else '[No relevant excerpts found]'}\n\n"
        f"VERIFICATION QUESTIONS:\n{ver_q_block}\n\n"
        "Output ONLY the answers, one per line in the same order."
    )
    ver_answers = bot.reply(ver_ans_prompt, max_tokens=256, temperature=0.2).strip()

    # 4) Final answer (only)
    final_prompt = (
        "You are evaluating a model. Produce the FINAL answer using ONLY the EXCERPTS. "
        "If something in the DRAFT is not supported by the verification answers, remove or correct it. "
        "Cite chunks like (Chunk 12). If the excerpts do not contain the answer, say you don't know.\n\n"
        f"EXCERPTS:\n{context if context else '[No relevant excerpts found]'}\n\n"
        f"QUESTION:\n{question}\n\n"
        f"DRAFT:\n{draft}\n\n"
        f"VERIFICATION ANSWERS:\n{ver_answers}\n\n"
        "Output ONLY the final answer."
    )
    return bot.reply(final_prompt, max_tokens=1024, temperature=0.2).strip()


def ask_on_pdf(question: str, *, k: int | None = None, multiquery: bool = True) -> str:
    if not ATTACHED_PDF:
        return bot.reply(question)
    k = RETRIEVE_K if k is None else k
    indices = retrieve_context_multiquery(question, k=k) if multiquery else retrieve_context(question, k=k)
    context = _build_context(indices)
    if COVE_ENABLED:
        return _cove_answer(question, context=context)
    prompt = (
        "You are evaluating a model. Answer the user's question using ONLY the provided document excerpts. "
        "Cite the chunks you used like (Chunk 12). If the excerpts do not contain the answer, say you don't know.\n\n"
        f"Document: {ATTACHED_PDF.name}\n\n"
        f"EXCERPTS:\n{context if context else '[No relevant excerpts found]'}\n\n"
        f"QUESTION:\n{question}"
    )
    return bot.reply(prompt, max_tokens=1024)


def _extract_first_upload(value) -> tuple[str, bytes] | None:
    """Return (filename, content_bytes) for the first uploaded file."""
    if not value:
        return None
    if isinstance(value, dict):
        first_key = next(iter(value.keys()))
        first = value[first_key]
        if isinstance(first, dict) and "content" in first:
            name = first.get("name") or first_key
            return name, first["content"]
        if isinstance(first, (bytes, bytearray)):
            return first_key, bytes(first)
    if isinstance(value, (tuple, list)):
        first = value[0]
        if isinstance(first, dict) and "content" in first:
            name = first.get("name") or "uploaded.pdf"
            return name, first["content"]
        name = getattr(first, "name", "uploaded.pdf")
        content = getattr(first, "content", None)
        if content is None:
            content = getattr(first, "data", None)
        if content is None and isinstance(first, (bytes, bytearray)):
            content = bytes(first)
        if isinstance(content, (bytes, bytearray)):
            return name, bytes(content)
    name = getattr(value, "name", "uploaded.pdf")
    content = getattr(value, "content", None)
    if isinstance(content, (bytes, bytearray)):
        return name, bytes(content)
    return None


def show_pdf_uploader() -> None:
    if widgets is None or display is None:
        print("ipywidgets not available in this environment. Use attach_pdf_text(load_pdf_from_path(...))")
        return
    uploader = widgets.FileUpload(accept=".pdf", multiple=False)
    button = widgets.Button(description="Load PDF")
    out = widgets.Output()

    def _on_click(_):
        with out:
            out.clear_output()
            item = _extract_first_upload(uploader.value)
            if not item:
                print("Select a PDF first")
                return
            name, data = item
            text = _pdf_bytes_to_text(data)
            attach_pdf_text(name, text)

    button.on_click(_on_click)
    display(widgets.VBox([widgets.HTML("<b>Attach a PDF</b>"), uploader, button, out]))


# Run this to attach a PDF via UI (if supported):
# show_pdf_uploader()

# Or attach from a local path (Windows example):
# attach_pdf_text("my.pdf", load_pdf_from_path(r"C:\\path\\to\\file.pdf"))

c:\Users\celli\AppData\Local\Programs\Python\Python310\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
show_pdf_uploader()

In [8]:
import json
import sys
import time
from typing import Any, Dict, List, Optional, Tuple


def _print(*args, **kwargs) -> None:
    """print(..., flush=True) by default (important for VS Code notebook input loops)."""
    kwargs.setdefault("flush", True)
    print(*args, **kwargs)


def _extract_json_array(text: str) -> list[dict[str, Any]]:
    """Best-effort: find and parse a JSON array from a model response."""
    s = (text or "").strip()
    if not s:
        raise ValueError("Empty model output")
    # Fast path: exact JSON
    try:
        obj = json.loads(s)
        if isinstance(obj, list):
            return obj  # type: ignore[return-value]
    except Exception:
        pass
    # Slow path: extract the first [...] block
    start = s.find("[")
    end = s.rfind("]")
    if start >= 0 and end > start:
        candidate = s[start : end + 1]
        obj = json.loads(candidate)
        if isinstance(obj, list):
            return obj  # type: ignore[return-value]
    raise ValueError("Could not parse JSON array from model output")


def generate_colregs_quiz(*, n: int = 3) -> List[Dict[str, Any]]:
    """Generate a short COLREGS quiz (JSON objects). Requires an attached PDF."""
    if not ATTACHED_PDF:
        raise RuntimeError("No PDF attached. Attach COLREGS PDF first.")
    bot.reset()
    prompt = (
        "You are generating a short quiz to test COLREGS knowledge. "
        "Generate EXACTLY 3 questions. Output ONLY valid JSON: a JSON array of 3 objects.\n\n"
        "Each object must have:\n"
        "- question: string (ask about a specific rule/definition/situation)\n"
        "- rule_refs: array of strings (e.g., ['Rule 5','Rule 7'] or ['Annex IV'])\n"
        "- ideal_answer: string (1-3 sentences, no chain-of-thought)\n\n"
        "Constraints:\n"
        "- Every question must be clearly answerable from COLREG text (avoid trick questions)\n"
        "- Keep each question <= 35 words\n"
        "- Mix difficulty: 1 easy, 1 medium, 1 harder\n"
        "- Do NOT mention fatigue or tiredness\n"
        "- Do NOT add extra keys\n"
    )
    raw = bot.reply(prompt, max_tokens=600, temperature=0.4)
    quiz = _extract_json_array(raw)
    quiz = quiz[:n]
    out: List[Dict[str, Any]] = []
    for item in quiz:
        if not isinstance(item, dict):
            continue
        q = str(item.get("question", "")).strip()
        if not q:
            continue
        rule_refs = item.get("rule_refs", [])
        if not isinstance(rule_refs, list):
            rule_refs = []
        rule_refs = [str(x).strip() for x in rule_refs if str(x).strip()]
        ideal = str(item.get("ideal_answer", "")).strip()
        out.append({"question": q, "rule_refs": rule_refs, "ideal_answer": ideal})
    if len(out) != n:
        raise RuntimeError(f"Quiz generation failed (got {len(out)} questions). Try again.")
    return out


def _grade_one(*, question: str, user_answer: str, ideal_answer: str, rule_refs: List[str]) -> Dict[str, Any]:
    """Grade a single answer using retrieved excerpts. Returns {score, verdict, feedback}."""
    if not ATTACHED_PDF:
        raise RuntimeError("No PDF attached")
    # Retrieve based on question and the ideal answer (helps anchor retrieval).
    retrieval_query = question if not ideal_answer else f"{question}\nExpected: {ideal_answer}"
    indices = retrieve_context_multiquery(retrieval_query, k=min(8, RETRIEVE_K))
    context = _build_context(indices)
    refs = ", ".join(rule_refs) if rule_refs else "(not specified)"
    prompt = (
        "You are grading a student's COLREGS answer using ONLY the EXCERPTS.\n"
        "Return ONLY valid JSON with keys: score, verdict, feedback.\n\n"
        "Scoring rules:\n"
        "- score must be 0, 0.5, or 1\n"
        "- verdict must be one of: 'incorrect', 'partial', 'correct'\n"
        "- feedback must be 1-3 short sentences and may cite chunks like (Chunk 12)\n\n"
        f"Target rule refs (hint only): {refs}\n\n"
        f"EXCERPTS:\n{context if context else '[No relevant excerpts found]'}\n\n"
        f"QUESTION:\n{question}\n\n"
        f"IDEAL ANSWER (short):\n{ideal_answer}\n\n"
        f"STUDENT ANSWER:\n{user_answer}\n\n"
    )
    raw = bot.reply(prompt, max_tokens=400, temperature=0.2)
    try:
        obj = json.loads(raw)
    except Exception:
        # best-effort extraction
        start = (raw or "").find("{")
        end = (raw or "").rfind("}")
        obj = json.loads(raw[start : end + 1]) if (start >= 0 and end > start) else {}
    score = obj.get("score", 0)
    verdict = obj.get("verdict", "incorrect")
    feedback = obj.get("feedback", "")
    if score not in (0, 0.5, 1):
        score = 0
    if verdict not in ("incorrect", "partial", "correct"):
        verdict = "incorrect"
    feedback = str(feedback).strip()
    return {"score": score, "verdict": verdict, "feedback": feedback}


def _fatigue_heuristic(total_score: float, *, max_score: float) -> str:
    """Non-medical heuristic: map performance to a fatigue hint."""
    # Keep it simple and deterministic (avoid the model going off-script).
    if max_score <= 0:
        return "Insufficient data to infer anything."
    pct = total_score / max_score
    if pct >= 0.85:
        return (
            "Your answers were strong and consistent. This suggests your COLREGS recall is currently solid. "
            "If you still feel tired, trust your body—this quiz is not a medical test."
        )
    if pct >= 0.55:
        return (
            "Your answers were mixed. That can happen from normal knowledge gaps or from fatigue. "
            "If this is unusual for you, consider taking a short break and re-trying later."
        )
    return (
        "You struggled to answer consistently. That can be caused by fatigue, stress, or simply unfamiliarity. "
        "If safety-critical, treat this as a signal to pause and recover rather than pushing on."
    )


def run_colregs_quiz() -> None:
    """Interactive 3-question quiz, then a short conclusion."""
    if not ATTACHED_PDF:
        _print("No PDF attached. Attach COLREGS PDF first (show_pdf_uploader() or attach_pdf_text(...)).")
        return
    try:
        quiz = generate_colregs_quiz(n=3)
    except Exception as e:
        _print(f"Quiz generation error: {e}")
        return
    _print("\n--- COLREGS Quiz (3 questions) ---")
    _print("Answer in your own words. Type '/exit' to stop the quiz.\n")
    total = 0.0
    max_total = float(len(quiz))
    for i, item in enumerate(quiz, start=1):
        q = item["question"]
        refs = item.get("rule_refs", [])
        refs_txt = ("; ".join(refs)) if refs else ""
        _print(f"Q{i}: {q}" + (f"  [{refs_txt}]" if refs_txt else ""))
        try:
            sys.stdout.flush()
            user_answer = input("Your answer> ").strip()
        except (EOFError, KeyboardInterrupt):
            _print("\n(quiz aborted)")
            return
        if user_answer.lower().strip() in {"/exit", "/quit"}:
            _print("(quiz aborted)")
            return
        try:
            graded = _grade_one(
                question=q,
                user_answer=user_answer,
                ideal_answer=item.get("ideal_answer", ""),
                rule_refs=refs,
            )
        except Exception as e:
            _print(f"Grading error: {e}")
            graded = {"score": 0, "verdict": "incorrect", "feedback": "Could not grade from excerpts."}
        total += float(graded.get("score", 0) or 0)
        verdict = graded.get("verdict", "incorrect")
        feedback = graded.get("feedback", "")
        _print(f"Result: {verdict} (score {graded.get('score', 0)}/1)")
        if feedback:
            _print(f"Feedback: {feedback}")
        _print("")
        time.sleep(0.2)
    _print("--- Quiz complete ---")
    _print(f"Total score: {total:.1f}/{max_total:.0f}")
    _print("Fatigue hint (non-medical):")
    _print(_fatigue_heuristic(total, max_score=max_total))
    _print("")


def chat_loop():
    """Interactive loop.

    Commands:
      /reset       clear chat history
      /exit        quit
      /pdf         show PDF status
      /clearpdf    detach current PDF
      /k N         set retrieval chunks (current default shown in /pdf)
      /quiz        run a 3-question COLREGS quiz + conclusion
    """
    global RETRIEVE_K
    _print(f"Connected to {getattr(bot, 'model', 'model')}. Type '/reset' or '/exit'.")
    _print("Use '/pdf' to show PDF status after attaching a PDF.")
    while True:
        try:
            sys.stdout.flush()
            user = input("You> ").strip()
        except (EOFError, KeyboardInterrupt):
            _print("\nBye")
            return
        if not user:
            continue
        low = user.lower().strip()
        if low in {"/exit", "/quit"}:
            _print("Bye")
            return
        if low == "/reset":
            bot.reset()
            _print("(history cleared)")
            continue
        if low == "/pdf":
            if ATTACHED_PDF:
                _print(
                    f"(pdf attached: {ATTACHED_PDF.name}, {len(ATTACHED_PDF.chunks)} chunks, "
                    f"k={RETRIEVE_K}, chunk_tokens={CHUNK_TOKENS}, overlap={CHUNK_OVERLAP_TOKENS}, "
                    f"bm25_w={BM25_WEIGHT}, sem_w={SEM_WEIGHT}, cove={COVE_ENABLED})"
                )
            else:
                _print("(no pdf attached) -> run show_pdf_uploader() or attach_pdf_text(...)")
            continue
        if low == "/clearpdf":
            clear_pdf()
            continue
        if low == "/quiz":
            run_colregs_quiz()
            continue
        if low == "/k" or low.startswith("/k "):
            parts = low.split()
            if len(parts) != 2:
                _print("Usage: /k 10")
                continue
            try:
                RETRIEVE_K = max(1, int(parts[1]))
                _print(f"(k set to {RETRIEVE_K})")
            except Exception:
                _print("Usage: /k 10")
            continue
        try:
            # Keep interactive chat responsive: avoid multiquery (extra LLM calls) by default here.
            answer = ask_on_pdf(user, multiquery=False) if ATTACHED_PDF else bot.reply(user)
        except Exception as e:
            _print(f"Error: {e}")
            continue
        _print(f"Bot> {answer}\n")


# Run interactive chat in the notebook output
# chat_loop()

In [143]:
chat_loop()

Connected to llama3.1:latest. Type '/reset' or '/exit'.
Use '/pdf' to show PDF status after attaching a PDF.
(pdf attached: COLREG-Consolidated-2018.pdf, 55 chunks, k=10, chunk_tokens=1024, overlap=256, bm25_w=0.55, sem_w=0.45, cove=True)

--- COLREGS Quiz (3 questions) ---
Answer in your own words. Type '/exit' to stop the quiz.

Q1: What is the minimum speed a vessel must have when navigating in a narrow channel?  [Rule 9]
Result: incorrect (score 0/1)

Q2: When overtaking another power-driven vessel, what must you do to avoid a collision?  [Rule 15]
Result: partial (score 0.5/1)
Feedback: The student correctly identified the overtaking vessel's responsibility but failed to mention giving at least two short blasts on the horn as required by Rule 13(b) and Rule 34(c)(i).

Q3: What is the responsibility of an overtaking vessel when encountering another vessel head-on?  [Rule 14]
Result: incorrect (score 0/1)
Feedback: The student incorrectly applied Rule 17(c) to a head-on situation, w

In [9]:
# --- Batch evaluation from CSV (RAG) ---
from pathlib import Path

import pandas as pd


def _find_existing_path(*candidates: str) -> Path:
    for c in candidates:
        p = Path(c)
        if p.exists():
            return p.resolve()
    raise FileNotFoundError(f"None of these paths exist: {candidates}")


def _read_csv_robust(path: Path) -> pd.DataFrame:
    try:
        return pd.read_csv(path)
    except UnicodeDecodeError:
        # Common fallbacks for Windows CSV exports
        for enc in ("utf-8-sig", "cp1252", "latin1"):
            try:
                return pd.read_csv(path, encoding=enc)
            except Exception:
                pass
        raise


# Notebook usually runs with CWD = DeepSeek_test/. These files are at the workspace root.
CSV_PATH = _find_existing_path(
    "Set évaluation RAG.csv",
    str(Path("..") / "Set évaluation RAG.csv"),
    str(Path("..") / ".." / "Set évaluation RAG.csv"),
    r"C:\Users\celli\Documents\.PIP2026\PIP-2026-LOTUSim_G5_Fatigue\Set évaluation RAG.csv",
)
PDF_PATH = _find_existing_path(
    "COLREG-Consolidated-2018.pdf",
    str(Path("..") / "COLREG-Consolidated-2018.pdf"),
    str(Path("..") / ".." / "COLREG-Consolidated-2018.pdf"),
    r"C:\Users\celli\Documents\.PIP2026\PIP-2026-LOTUSim_G5_Fatigue\COLREG-Consolidated-2018.pdf",
)

print("CSV:", CSV_PATH)
print("PDF:", PDF_PATH)

# Load CSV and keep only the Question column for now
df_questions = _read_csv_robust(CSV_PATH)
if "Question" not in df_questions.columns:
    raise KeyError(f"CSV must have a 'Question' column. Found: {list(df_questions.columns)}")
questions = df_questions["Question"].dropna().astype(str).tolist()
print(f"Loaded {len(questions)} questions")

# Attach the PDF (only if not already attached or if you want to force reload)
if ATTACHED_PDF is None or (ATTACHED_PDF.name != PDF_PATH.name):
    attach_pdf_text(PDF_PATH.name, load_pdf_from_path(str(PDF_PATH)))

CSV: C:\Users\celli\Documents\.PIP2026\PIP-2026-LOTUSim_G5_Fatigue\Set évaluation RAG.csv
PDF: C:\Users\celli\Documents\.PIP2026\PIP-2026-LOTUSim_G5_Fatigue\COLREG-Consolidated-2018.pdf
Loaded 12 questions
Attached PDF: COLREG-Consolidated-2018.pdf (104,429 chars, 55 chunks)
Chunking: ~1024 tokens, overlap ~256 tokens


In [10]:
import time
# Run the evaluation: ask each question on the attached PDF
MAX_QUESTIONS = None  # set to an int for a quick dry-run, e.g. 5

questions_to_run = questions if not MAX_QUESTIONS else questions[:MAX_QUESTIONS]
results = []
total = len(questions_to_run)
for i, q in enumerate(questions_to_run, start=1):
    bot.reset()  # avoid history leakage across questions
    strict_prompt = ""
    t0 = time.time()
    ans = ask_on_pdf(strict_prompt + q)
    print(f"q{i} : {time.time() - t0}s")
    results.append({"Question": q, "Answer": ans})
    if i % 10 == 0 or i == total:
        print(f"Done {i}/{total}")

df_results = pd.DataFrame(results)
df_results.head()

# Save results next to the notebook
OUT_PATH = Path("rag_eval_answers.csv").resolve()
df_results.to_csv(OUT_PATH, index=False)
print("Saved:", OUT_PATH)

q1 : 51.774319648742676s
q2 : 61.1369526386261s
q3 : 74.2618477344513s
q4 : 65.50696015357971s
q5 : 63.7942955493927s
q6 : 95.42311549186707s
q7 : 92.42543840408325s
q8 : 55.08546566963196s
q9 : 109.19661474227905s
q10 : 93.6745240688324s
Done 10/12
q11 : 99.64414310455322s
q12 : 98.1123435497284s
Done 12/12
Saved: C:\Users\celli\Documents\.PIP2026\PIP-2026-LOTUSim_G5_Fatigue\DeepSeek_test\rag_eval_answers.csv
